# County NAICS Files
After running the notebook, delete the python/data_raw folder
Years Prior to 2012 currently have a bug when specifying NAICS as one of the columns. Years Prior to 2012 also only have the totals for all sectors not each sector.
Zip level not available -> https://api.census.gov/data/2011/cbp/variables.html  
Didn't use 2017 since NAICS 2017 wasn't available in master_crrosswalk

In [1]:
import csv
import requests as r
import pandas as pd
import zipfile, io
import os
from tqdm import tqdm
import pathlib
import pandas as pd
import datetime
endyear = datetime.date.today().year
api_headers = {}
api_headers['x-api-key'] = '975f39a54e48438ceebf303d6018e34db212e804'

In [2]:
# Set a relative location to save the data from the request
repo_dir = pathlib.Path().cwd()
print(repo_dir)
#repo_dir = os.getcwd();
raw_data_dir = repo_dir / 'data_raw'
out_data_dir = raw_data_dir / 'BEA_Industry_Factors'

# Create new folders if needed
#state_data_dir = out_data_dir / 'state_level'
#if not state_data_dir.exists():
#    state_data_dir.mkdir(parents=True)
    
county_data_dir = out_data_dir / 'county_level'
if not county_data_dir.exists():
    county_data_dir.mkdir(parents=True)
    
#zip_data_dir = out_data_dir / 'zip_level'
#if not county_data_dir.exists():
#    county_data_dir.mkdir(parents=True)

/Users/Gauravpatil/Workspaces/ModelEarth/community-data/process/python/bea


In [3]:
# print(county_data_dir)

In [4]:
# Load the state FIPS codes key
state_fips = pd.read_csv('../../../us/id_lists/state_fips.csv', usecols=['Name', 'Postal Code', 'FIPS'])
state_fips = state_fips.head(50)  # <-- limit to only US states, not teritories

In [5]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_county_cbp(fips, state, years):
    count = 0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year >= 2000 and year <= 2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2003 and year <=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2008 and year <= 2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2012 and year <= 2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
        elif year >= 2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url = f"{base_url}/{year}/cbp?get={columns_to_select}&for=county:*&in=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(county_data_dir / f"industriesPerCounty_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [6]:
#TEMP endyear
endyear = 2021

In [7]:
for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     years=range(2012,endyear)
     get_county_cbp(fips, state, years)

Getting data for state: Alabama	year: 2012
  > Finished CSV for year2012
Getting data for state: Alabama	year: 2013
  > Finished CSV for year2013
Getting data for state: Alabama	year: 2014
  > Finished CSV for year2014
Getting data for state: Alabama	year: 2015
  > Finished CSV for year2015
Getting data for state: Alabama	year: 2016
  > Finished CSV for year2016
Getting data for state: Alabama	year: 2017
  > Finished CSV for year2017
Getting data for state: Alabama	year: 2018
  > Finished CSV for year2018
Getting data for state: Alabama	year: 2019
  > Finished CSV for year2019
Getting data for state: Alabama	year: 2020
  > Finished CSV for year2020
Getting data for state: Alaska	year: 2012
  > Finished CSV for year2012
Getting data for state: Alaska	year: 2013
  > Finished CSV for year2013
Getting data for state: Alaska	year: 2014
  > Finished CSV for year2014
Getting data for state: Alaska	year: 2015
  > Finished CSV for year2015
Getting data for state: Alaska	year: 2016
  > Finished 

# Data Aggregation
This part allows us to manage different Fips level (county/state) and different NAICS level (sector/industry/etc...)  
Trick: Getting the NAICS code from all the NAICS2012 files that we downloaded

In [8]:
# Load the 2012 data

def load_all_states2012(bea_data_dir):
    
    for i in range(2012,endyear):
        #print(i)
        x="_"+str(i)
        files = [f for f in bea_data_dir.iterdir() if x in f.name]

        for f in files:
            #print(f)
            # variable slecetion based on census year
            naics_str = "NAICS2012" if i < 2017 else "NAICS2017"
            naics_ttl = "NAICS2012_TTL" if i < 2017 else "NAICS2017_LABEL"
            geo_ttl = "GEO_TTL" if i < 2017 else "NAME"

            #df = pd.read_csv(f,encoding='latin-1',dtype={"NAICS2012": str})
            df = pd.read_csv(f,encoding='latin-1',dtype={naics_str: str})
            if 'Unnamed: 11' in df.columns:
                df=df.drop("Unnamed: 11", axis=1)
            if 'Unnamed: 10' in df.columns:
                df=df.drop("Unnamed: 10", axis=1)

            # renaming columns so similar data from 2012 census & 2017 census are entered into appropriate columns
            df = df.rename(columns={"fips": "id", naics_str: "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab", naics_ttl:"NAICS_TTL", geo_ttl:"GEO_TTL"})
            naics_str = "relevant_naics"
            naics_ttl = "NAICS_TTL"
            geo_ttl = "GEO_TTL"
            #df = pd.read_csv(f)
            #df[df['NAICS2012'].str.len() != 5]
            #b=df[df['NAICS2012'].str.len() == 5]
            #df['is5'] = df['NAICS2012'].apply(lambda x: 'True' if len(x) == 5 else 'False')
            df['is5'] = df[naics_str].apply(lambda x: 'True' if len(x) == 5 else 'False')
                #print("hh"+df.NAICS2012)
                #print("hghg"+len(df.NAICS2012))
            #df['NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2] if df['is5']=='False' else 1).apply(int)
            # df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df['NAICS2012']
            # df.loc[(df['is5'] == 'True') & (df['NAICS2012'].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
            # df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df['NAICS2012'].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) == '-'), 'NAICS_Sector'] = df[naics_str]
            df.loc[(df['is5'] == 'True') & (df[naics_str].apply(lambda v: v[2:3]) != '-'), 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])
            df.loc[(df['is5'] == 'False') , 'NAICS_Sector'] = df[naics_str].apply(lambda v: v[:2])
                #print("hgh"+df.NAICS2012[1:2])
                #if (df.NAICS2012.apply(lambda v: v[0:1]).apply(int)==1).bool():
                #    print("bye")

            yield df
    
df = pd.concat(load_all_states2012(county_data_dir)).drop("is5", axis=1)

# test = list(load_all_states(bea_data_dir))

# df

In [9]:
df=df.drop("county", axis=1)

### Process FIPS Code
FIPS is the federal/census unique ID for each geographic area.  States have 2 digives and counties have 5

In [10]:
# Process FIPS code
df['fips'] = df.GEO_ID.apply(lambda GID: GID.split('US')[1])
# # Remove the state_level data
# df_state = df[df['fips'].str.len() == 2]
# df_county = df[df['fips'].str.len() != 2]

def county_level(df):
    return df[df['id'].str.len() == 5]

def state_level(df):
    return df[df['id'].str.len() == 2]

In [11]:
# df.to_csv("allll.csv")

### Group all data per sector for  every year and average them  
Does it make sense to sum up instead of average?!

In [12]:
naics_str = "relevant_naics"
naics_ttl = "NAICS_TTL"
geo_ttl = "GEO_TTL"

newDF=df.groupby(['fips','state','COUNTY',geo_ttl, naics_str, naics_ttl,'NAICS_Sector', 'GEO_ID'],as_index=False).mean()
newDF=newDF.drop("YEAR", axis=1)

In [13]:
#newDF=newDF.astype({'fips': 'int'})
# newDF=newDF.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
newDF=newDF.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
0,01001,1,1,Autauga County,00,Total for all sectors,00,0500000US01001,817.00,9908.0,280188.0
1,01001,1,1,Autauga County,11,"Agriculture, forestry, fishing and hunting",11,0500000US01001,6.00,30.0,1102.0
2,01001,1,1,Autauga County,113,Forestry and logging,11,0500000US01001,6.00,30.0,1102.0
3,01001,1,1,Autauga County,1133,Logging,11,0500000US01001,6.00,30.0,1102.0
4,01001,1,1,Autauga County,11331,Logging,11,0500000US01001,6.00,30.0,1102.0
...,...,...,...,...,...,...,...,...,...,...,...
4435267,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,0500000US56045,1.00,0.0,0.0
4435268,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,0500000US56045,1.00,0.0,0.0
4435269,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,0500000US56045,1.00,0.0,0.0
4435270,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,0500000US56045,1.00,0.0,0.0


In [14]:
newDF.tail(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
4435222,56045,56,45,"Weston County, Wyoming",7139,Other amusement and recreation industries,71,0500000US56045,2.000000,0.000,0.000000
4435223,56045,56,45,"Weston County, Wyoming",71391,Golf courses and country clubs,71,0500000US56045,2.000000,0.000,0.000000
4435224,56045,56,45,"Weston County, Wyoming",713910,Golf courses and country clubs,71,0500000US56045,2.000000,0.000,0.000000
4435225,56045,56,45,"Weston County, Wyoming",72,Accommodation and food services,72,0500000US56045,23.625000,188.875,2210.000000
4435226,56045,56,45,"Weston County, Wyoming",721,Accommodation,72,0500000US56045,7.125000,35.875,677.250000
4435227,56045,56,45,"Weston County, Wyoming",7211,Traveler accommodation,72,0500000US56045,6.250000,35.375,656.875000
4435228,56045,56,45,"Weston County, Wyoming",72111,Hotels (except casino hotels) and motels,72,0500000US56045,6.250000,35.375,656.875000
4435229,56045,56,45,"Weston County, Wyoming",721110,Hotels (except casino hotels) and motels,72,0500000US56045,6.250000,35.375,656.875000
4435230,56045,56,45,"Weston County, Wyoming",722,Food services and drinking places,72,0500000US56045,16.500000,135.750,1426.000000
4435231,56045,56,45,"Weston County, Wyoming",7224,Drinking places (alcoholic beverages),72,0500000US56045,2.875000,7.625,68.375000


### Group data by NAICS Sector

NAICS is the North American Industry Classification System. The coarsest level of classification is the *Sector*.

The organization of NAICS is as follows:  <-- from [this page](https://www.census.gov/programs-surveys/economic-census/guidance/understanding-naics.html) on census.gov
- Sector: 2-digit code
    - Subsector: 3-digit code
        - Industry Group: 4-digit code
            - NAICS Industry: 5-digit code
                - National Industry: 6-digit code

Start by grouping the data by sector:

In [15]:
def naics_level(df, naics_level):
    return df[df['relevant_naics'].str.len() == naics_level]

In [16]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2 = naics_level(newDF, 2).reset_index(drop=True)
df_naics_3 = naics_level(newDF, 3).reset_index(drop=True)
df_naics_4 = naics_level(newDF, 4).reset_index(drop=True)
df_naics_5 = naics_level(newDF, 5).reset_index(drop=True)
df_naics_6 = naics_level(newDF, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2 = df_naics_2[df_naics_2.relevant_naics != '00']
df_naics_3 = df_naics_3[df_naics_3.relevant_naics != '00']
df_naics_4 = df_naics_4[df_naics_4.relevant_naics != '00']
df_naics_5 = df_naics_5[df_naics_5.relevant_naics != '00']
df_naics_6 = df_naics_6[df_naics_6.relevant_naics != '00']

In [17]:
#s2=state_level(df_naics_2)
c2=county_level(df_naics_2)
#s3=state_level(df_naics_3)
#c3=county_level(df_naics_3)
#s4=state_level(df_naics_4)
c4=county_level(df_naics_4)
#s5=state_level(df_naics_5)
#c5=county_level(df_naics_5)
s6=state_level(df_naics_6)
#c6=county_level(df_naics_6)

In [18]:
#s2.to_csv('s2.tsv',sep='\t')
#c2.to_csv('c2.tsv',sep='\t')
#s3.to_csv('s3.tsv',sep='\t')
#c3.to_csv('c3.tsv',sep='\t')
#s4.to_csv('s4.tsv',sep='\t')
#c4.to_csv('c4.tsv',sep='\t')
#s5.to_csv('s5.tsv',sep='\t')
#c5.to_csv('c5.tsv',sep='\t')
#s6.to_csv('s6.tsv',sep='\t')
#c6.to_csv('c6.tsv',sep='\t')

In [19]:
newDF

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
0,01001,1,1,Autauga County,00,Total for all sectors,00,0500000US01001,817.00,9908.0,280188.0
1,01001,1,1,Autauga County,11,"Agriculture, forestry, fishing and hunting",11,0500000US01001,6.00,30.0,1102.0
2,01001,1,1,Autauga County,113,Forestry and logging,11,0500000US01001,6.00,30.0,1102.0
3,01001,1,1,Autauga County,1133,Logging,11,0500000US01001,6.00,30.0,1102.0
4,01001,1,1,Autauga County,11331,Logging,11,0500000US01001,6.00,30.0,1102.0
...,...,...,...,...,...,...,...,...,...,...,...
4435267,56045,56,45,"Weston County, Wyoming",81391,Business associations,81,0500000US56045,1.00,0.0,0.0
4435268,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,0500000US56045,1.00,0.0,0.0
4435269,56045,56,45,"Weston County, Wyoming",81393,Labor unions and similar labor organizations,81,0500000US56045,1.00,0.0,0.0
4435270,56045,56,45,"Weston County, Wyoming",813930,Labor unions and similar labor organizations,81,0500000US56045,1.00,0.0,0.0


In [20]:
#newDF.to_csv('all.tsv',sep='\t')

# The statewide data does not include NAICS starting with 1!

In [21]:
a=county_level(df_naics_2)

In [22]:
county_level(df_naics_6)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
0,01001,1,1,Autauga County,113310,Logging,11,0500000US01001,6.000,30.00,1102.0
1,01001,1,1,Autauga County,212319,Other crushed and broken stone mining and quar...,21,0500000US01001,1.000,0.00,0.0
2,01001,1,1,Autauga County,212321,Construction sand and gravel mining,21,0500000US01001,1.000,0.00,0.0
3,01001,1,1,Autauga County,221112,Fossil fuel electric power generation,22,0500000US01001,4.000,0.00,0.0
4,01001,1,1,Autauga County,221122,Electric power distribution,22,0500000US01001,4.000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1656018,56045,56,45,"Weston County, Wyoming",813110,Religious organizations,81,0500000US56045,8.375,14.75,264.5
1656019,56045,56,45,"Weston County, Wyoming",813312,"Environment, conservation and wildlife organiz...",81,0500000US56045,1.000,0.00,0.0
1656020,56045,56,45,"Weston County, Wyoming",813319,Other social advocacy organizations,81,0500000US56045,1.000,0.00,0.0
1656021,56045,56,45,"Weston County, Wyoming",813910,Business associations,81,0500000US56045,1.000,0.00,0.0


In [23]:
a=df_naics_6[df_naics_6.id=='13001'].sort_values(by=['payann'],ascending=False).head(50)
#a.to_csv("investigate.csv")

# NAICS code to name translation

In [24]:
NAICS_codes = pd.read_csv('../../../us/Crosswalk_MasterCrosswalk.csv', usecols=['2012_NAICS_Code', '2012_NAICS_Industry'])

In [25]:
NAICS_codes=NAICS_codes.rename(columns={"2012_NAICS_Code": "relevant_naics", "2012_NAICS_Industry": "industry_detail"})

In [26]:
NAICS_codes=NAICS_codes.dropna()

In [27]:
NAICS_codes=NAICS_codes.drop_duplicates()

In [28]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
13,111.0,Crop Production
19,1111.0,Oilseed and Grain Farming
21,11111.0,Soybean Farming
22,111110.0,Soybean Farming
...,...,...
3885,9281.0,National Security and International Affairs
3887,92811.0,National Security
3888,928110.0,National Security
3889,92812.0,International Affairs


In [29]:
#adding the row for Industries not classified
NAICS_codes
new_row = {'relevant_naics':99, 'industry_detail':"Industries not classified"}
#append row to the dataframe
# NAICS_codes = NAICS_codes.append(new_row, ignore_index=True)
NAICS_codes = pd.concat([NAICS_codes, pd.DataFrame([new_row])], ignore_index=True)

In [30]:
NAICS_codes

,relevant_naics,industry_detail
0,11.0,"Agriculture, Forestry, Fishing and Hunting"
1,111.0,Crop Production
2,1111.0,Oilseed and Grain Farming
3,11111.0,Soybean Farming
4,111110.0,Soybean Farming
...,...,...
2210,92811.0,National Security
2211,928110.0,National Security
2212,92812.0,International Affairs
2213,928120.0,International Affairs


In [31]:
NAICS_codes.to_csv('../../../us/id_lists/industry_ID_list.csv')

# Making a states json

In [32]:
stateFips = pd.read_csv('../../../us/id_lists/state_fips.csv')

In [33]:
stateFips=stateFips.drop(['Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6'],axis=1)

In [34]:
#stateFips=stateFips.set_index('Name')

In [35]:
stateFips

,Name,Postal Code,FIPS
0,Alabama,AL,1
1,Alaska,AK,2
2,Arizona,AZ,4
3,Arkansas,AR,5
4,California,CA,6
5,Colorado,CO,8
6,Connecticut,CT,9
7,Delaware,DE,10
8,Florida,FL,12
9,Georgia,GA,13


In [36]:
#stateFips.to_json(county_data_dir/'states.json', orient = "records", date_format = "epoch", double_precision = 10, force_ascii = True, date_unit = "ms", default_handler = None)

# Making county to fips csv

In [37]:
countyDF=c2[['GEO_TTL','id']].drop_duplicates()

In [38]:
countyDF

,GEO_TTL,id
1,Autauga County,01001
19,"Autauga County, Alabama",01001
37,Baldwin County,01003
54,"Baldwin County, Alabama",01003
72,Barbour County,01005
...,...,...
104216,"Uinta County, Wyoming",56041
104233,Washakie County,56043
104250,"Washakie County, Wyoming",56043
104267,Weston County,56045


In [39]:
countyDF['hascomma'] =countyDF['GEO_TTL'].apply(lambda x: 'True' if ',' in x else 'False')
countyDF
#countyDF['county'] = df.GEO_TTL.apply(lambda GTT: GTT.split(',')[0])

,GEO_TTL,id,hascomma
1,Autauga County,01001,False
19,"Autauga County, Alabama",01001,True
37,Baldwin County,01003,False
54,"Baldwin County, Alabama",01003,True
72,Barbour County,01005,False
...,...,...,...
104216,"Uinta County, Wyoming",56041,True
104233,Washakie County,56043,False
104250,"Washakie County, Wyoming",56043,True
104267,Weston County,56045,False


In [40]:
countyDF.loc[(countyDF['hascomma'] == 'True'), 'county'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[0])
countyDF.loc[(countyDF['hascomma'] == 'True'), 'state'] = countyDF.GEO_TTL.apply(lambda GTT: GTT.split(', ')[-1])

In [41]:
countyDF=countyDF[['state','county','id']].drop_duplicates()

In [42]:
countyDF = countyDF.dropna()

In [43]:
countyDF

,state,county,id
19,Alabama,Autauga County,01001
54,Alabama,Baldwin County,01003
89,Alabama,Barbour County,01005
123,Alabama,Bibb County,01007
159,Alabama,Blount County,01009
...,...,...,...
104145,Wyoming,Sweetwater County,56037
104181,Wyoming,Teton County,56039
104216,Wyoming,Uinta County,56041
104250,Wyoming,Washakie County,56043


In [44]:
stats=stateFips.rename(columns={"Name": "state"})

In [45]:
stats=stats.drop("FIPS",axis=1)
stats

,state,Postal Code
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA
5,Colorado,CO
6,Connecticut,CT
7,Delaware,DE
8,Florida,FL
9,Georgia,GA


In [46]:
countyDF=countyDF.merge(stats, on='state', how='left')

In [47]:
countyDF=countyDF.rename(columns={"Postal Code": "abvr"})
countyDF

,state,county,id,abvr
0,Alabama,Autauga County,01001,AL
1,Alabama,Baldwin County,01003,AL
2,Alabama,Barbour County,01005,AL
3,Alabama,Bibb County,01007,AL
4,Alabama,Blount County,01009,AL
...,...,...,...,...
3140,Wyoming,Sweetwater County,56037,WY
3141,Wyoming,Teton County,56039,WY
3142,Wyoming,Uinta County,56041,WY
3143,Wyoming,Washakie County,56043,WY


countyDF.to_csv('../../../us/id_lists/county_ID_list.csv')

# Saving the County level data

In [48]:
states=newDF.state.unique()
#states=[13]
df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
a=county_level(df_naics_2)
b=county_level(df_naics_4)
c=county_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)

    repo_dir = pathlib.Path().cwd()
    state_dir = repo_dir.parents[2] / 'us' / 'state-new' / stateName
    
    if not state_dir.exists():
        state_dir.mkdir(parents=True)
        
    a1=a[a.state==state]
    a1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics2.tsv",sep='\t')
    
    b1=b[b.state==state]
    b1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics4.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics6.tsv",sep='\t')
    #print(c1.dtypes)

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


# saving the statewide data

In [49]:
states=newDF.state.unique()
#states=[13]



for state in states:
    b1=county_level(df_naics_2)
    c1=b1[b1.state==state]
    c1.astype({'NAICS_Sector': 'int'})
    # d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d1=c1.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d1.insert(0, 'id', state)
    d1.insert(1, 'COUNTY', 999)
    d1.insert(2, 'GEO_TTL', 'Statewide')

    b2=county_level(df_naics_4)
    c2=b2[b2.state==state]
    c2.astype({'NAICS_Sector': 'int'})
    # d2=c2.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d2=c2.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d2.insert(0, 'id', state)
    d2.insert(1, 'COUNTY', 999)
    d2.insert(2, 'GEO_TTL', 'Statewide')

    b3=county_level(df_naics_6)
    c3=b3[b3.state==state]
    c3.astype({'NAICS_Sector': 'int'})
    # d3=c3.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d3=c3.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY","id", "GEO_TTL"],axis=1)
    d3.insert(0, 'id', state)
    d3.insert(1, 'COUNTY', 999)
    d3.insert(2, 'GEO_TTL', 'Statewide')
    
    
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a11=d1[d1.state==state]
    a11.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics2_state.tsv",sep='\t')
    #print(a11)
    #print(a11.dtypes)
    
    b11=d2[d2.state==state]
    b11.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics4_state.tsv",sep='\t')
    
    c11=d3[d3.state==state]
    c11.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics6_state.tsv",sep='\t')

AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


### statewide from API, the wrong version!

In [50]:
"""
states=newDF.state.unique()
#states=[13]
a=state_level(df_naics_2)
b=state_level(df_naics_4)
c=state_level(df_naics_6)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state_api.tsv",sep='\t')
    #print(a1)
    #print(a1.dtypes)
    
    b1=b[b.state==state]
    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state_api.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state_api.tsv",sep='\t')
"""

'\nstates=newDF.state.unique()\n#states=[13]\na=state_level(df_naics_2)\nb=state_level(df_naics_4)\nc=state_level(df_naics_6)\nfor state in states:\n    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]\n    print(stateName)\n    a1=a[a.state==state]\n    a1.to_csv(f"usa/{stateName}/industries_state{state}_naics2_state_api.tsv",sep=\'\t\')\n    #print(a1)\n    #print(a1.dtypes)\n    \n    b1=b[b.state==state]\n    b1.to_csv(f"usa/{stateName}/industries_state{state}_naics4_state_api.tsv",sep=\'\t\')\n    \n    c1=c[c.state==state]\n    c1.to_csv(f"usa/{stateName}/industries_state{state}_naics6_state_api.tsv",sep=\'\t\')\n'

### statewide data from API, The correct version

In [51]:
state_data_dir = out_data_dir / 'state_level'
if not state_data_dir.exists():
    state_data_dir.mkdir(parents=True)

In [52]:
# Base URL for the API call
base_url = "https://api.census.gov/data"

#
# NOTE Years Prior to 2012 Currently have a bug when specifying NAICS#### as one of the columns
#      - stick to 2012 and later for now
#


def get_state_cbp(fips, state, years):
    count=0
    for year in years:
        print(f"Getting data for state: {state}\tyear: {year}")
        if year>=2000 and year<=2002:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS1997_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2003 and year<=2007:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2002_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2008 and year<=2011:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2007_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year>=2012 and year<=2016:
            columns_to_select = "GEO_ID,GEO_TTL,COUNTY,YEAR,NAICS2012,NAICS2012_TTL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
        elif year==2017:
            columns_to_select = "GEO_ID,NAME,COUNTY,YEAR,NAICS2017,NAICS2017_LABEL,ESTAB,EMP,PAYANN"
            url=f"{base_url}/{year}/cbp?get={columns_to_select}&for=state:{fips:02d}"
    
    
        response = r.get(url, headers=api_headers)

        with open(state_data_dir / f"industriesPerState_{str.lower(state.replace(' ', ''))}_{year}.csv",'w') as resultPath:
            for line in response.text.strip().split('\n'):
                line=line.replace('[',"").replace(']',"")
                resultPath.write(line + "\n")

        print("  > Finished CSV for year"+str(year))

In [53]:
endyear = 2021

for fips in state_fips.FIPS.unique():
     state = state_fips.query(f'FIPS=={fips}').values[0][0]
     #years=range(2012,2018)
     years=range(2012,endyear)
     get_state_cbp(fips, state, years)

Getting data for state: Alabama	year: 2012
  > Finished CSV for year2012
Getting data for state: Alabama	year: 2013
  > Finished CSV for year2013
Getting data for state: Alabama	year: 2014
  > Finished CSV for year2014
Getting data for state: Alabama	year: 2015
  > Finished CSV for year2015
Getting data for state: Alabama	year: 2016
  > Finished CSV for year2016
Getting data for state: Alabama	year: 2017
  > Finished CSV for year2017
Getting data for state: Alabama	year: 2018
  > Finished CSV for year2018
Getting data for state: Alabama	year: 2019
  > Finished CSV for year2019
Getting data for state: Alabama	year: 2020
  > Finished CSV for year2020
Getting data for state: Alaska	year: 2012
  > Finished CSV for year2012
Getting data for state: Alaska	year: 2013
  > Finished CSV for year2013
Getting data for state: Alaska	year: 2014
  > Finished CSV for year2014
Getting data for state: Alaska	year: 2015
  > Finished CSV for year2015
Getting data for state: Alaska	year: 2016
  > Finished 

In [54]:
df_state = pd.concat(load_all_states2012(state_data_dir)).drop("is5", axis=1)

In [55]:
df_state

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector
0,0400000US01,Alabama,NaN,2012,00,Total for all sectors,97938,1585761,61284931,1,00
1,0400000US01,Alabama,NaN,2012,11,"Agriculture, forestry, fishing and hunting",785,6028,212695,1,11
2,0400000US01,Alabama,NaN,2012,113,Forestry and logging,576,4184,151238,1,11
3,0400000US01,Alabama,NaN,2012,114210,Hunting and trapping,6,50,1600,1,11
4,0400000US01,Alabama,NaN,2012,238140,Masonry contractors,174,1548,50764,1,23
...,...,...,...,...,...,...,...,...,...,...,...
1882,0400000US26,Michigan,NaN,2017,811490,Other personal and household goods repair and ...,587,1455,50226,26,81
1883,0400000US26,Michigan,NaN,2017,812111,Barber shops,127,477,9255,26,81
1884,0400000US26,Michigan,NaN,2017,812112,Beauty salons,2231,13425,276896,26,81
1885,0400000US26,Michigan,NaN,2017,81221,Funeral homes and funeral services,550,5098,142816,26,81


In [56]:
df_state['fips'] = df_state.GEO_ID.apply(lambda GID: GID.split('US')[1])
df_state

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,fips
0,0400000US01,Alabama,NaN,2012,00,Total for all sectors,97938,1585761,61284931,1,00,01
1,0400000US01,Alabama,NaN,2012,11,"Agriculture, forestry, fishing and hunting",785,6028,212695,1,11,01
2,0400000US01,Alabama,NaN,2012,113,Forestry and logging,576,4184,151238,1,11,01
3,0400000US01,Alabama,NaN,2012,114210,Hunting and trapping,6,50,1600,1,11,01
4,0400000US01,Alabama,NaN,2012,238140,Masonry contractors,174,1548,50764,1,23,01
...,...,...,...,...,...,...,...,...,...,...,...,...
1882,0400000US26,Michigan,NaN,2017,811490,Other personal and household goods repair and ...,587,1455,50226,26,81,26
1883,0400000US26,Michigan,NaN,2017,812111,Barber shops,127,477,9255,26,81,26
1884,0400000US26,Michigan,NaN,2017,812112,Beauty salons,2231,13425,276896,26,81,26
1885,0400000US26,Michigan,NaN,2017,81221,Funeral homes and funeral services,550,5098,142816,26,81,26


In [57]:
df_state["COUNTY"] = 0
df_state

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,fips
0,0400000US01,Alabama,0,2012,00,Total for all sectors,97938,1585761,61284931,1,00,01
1,0400000US01,Alabama,0,2012,11,"Agriculture, forestry, fishing and hunting",785,6028,212695,1,11,01
2,0400000US01,Alabama,0,2012,113,Forestry and logging,576,4184,151238,1,11,01
3,0400000US01,Alabama,0,2012,114210,Hunting and trapping,6,50,1600,1,11,01
4,0400000US01,Alabama,0,2012,238140,Masonry contractors,174,1548,50764,1,23,01
...,...,...,...,...,...,...,...,...,...,...,...,...
1882,0400000US26,Michigan,0,2017,811490,Other personal and household goods repair and ...,587,1455,50226,26,81,26
1883,0400000US26,Michigan,0,2017,812111,Barber shops,127,477,9255,26,81,26
1884,0400000US26,Michigan,0,2017,812112,Beauty salons,2231,13425,276896,26,81,26
1885,0400000US26,Michigan,0,2017,81221,Funeral homes and funeral services,550,5098,142816,26,81,26


In [58]:
naics_str = "relevant_naics"
naics_ttl = "NAICS_TTL"
geo_ttl = "GEO_TTL"

newDF_state=df_state.groupby(['fips','state','COUNTY',geo_ttl, naics_str, naics_ttl,'NAICS_Sector', "GEO_ID"],as_index=False).mean()
newDF_state=newDF_state.drop("YEAR", axis=1)
newDF_state

,fips,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
0,01,1,0,Alabama,00,Total for all sectors,00,0400000US01,99225.555556,1.651196e+06,6.792348e+07
1,01,1,0,Alabama,11,"Agriculture, forestry, fishing and hunting",11,0400000US01,793.222222,6.362556e+03,2.458331e+05
2,01,1,0,Alabama,113,Forestry and logging,11,0400000US01,576.000000,4.239778e+03,1.708111e+05
3,01,1,0,Alabama,1131,Timber tract operations,11,0400000US01,48.111111,2.892222e+02,1.373489e+04
4,01,1,0,Alabama,11311,Timber tract operations,11,0400000US01,48.111111,2.892222e+02,1.373489e+04
...,...,...,...,...,...,...,...,...,...,...,...
96751,56,56,0,Wyoming,81394,Political organizations,81,0400000US56,7.777778,1.488889e+01,6.516667e+02
96752,56,56,0,Wyoming,813940,Political organizations,81,0400000US56,7.777778,1.488889e+01,6.516667e+02
96753,56,56,0,Wyoming,81399,"Other similar organizations (except business, ...",81,0400000US56,19.888889,6.200000e+01,2.982222e+03
96754,56,56,0,Wyoming,813990,"Other similar organizations (except business, ...",81,0400000US56,19.888889,6.200000e+01,2.982222e+03


In [59]:
#newDF=newDF.astype({'fips': 'int'})
# newDF_state=newDF_state.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
newDF_state=newDF_state.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
newDF_state

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
0,01,1,0,Alabama,00,Total for all sectors,00,0400000US01,99225.555556,1.651196e+06,6.792348e+07
1,01,1,0,Alabama,11,"Agriculture, forestry, fishing and hunting",11,0400000US01,793.222222,6.362556e+03,2.458331e+05
2,01,1,0,Alabama,113,Forestry and logging,11,0400000US01,576.000000,4.239778e+03,1.708111e+05
3,01,1,0,Alabama,1131,Timber tract operations,11,0400000US01,48.111111,2.892222e+02,1.373489e+04
4,01,1,0,Alabama,11311,Timber tract operations,11,0400000US01,48.111111,2.892222e+02,1.373489e+04
...,...,...,...,...,...,...,...,...,...,...,...
96751,56,56,0,Wyoming,81394,Political organizations,81,0400000US56,7.777778,1.488889e+01,6.516667e+02
96752,56,56,0,Wyoming,813940,Political organizations,81,0400000US56,7.777778,1.488889e+01,6.516667e+02
96753,56,56,0,Wyoming,81399,"Other similar organizations (except business, ...",81,0400000US56,19.888889,6.200000e+01,2.982222e+03
96754,56,56,0,Wyoming,813990,"Other similar organizations (except business, ...",81,0400000US56,19.888889,6.200000e+01,2.982222e+03


In [60]:
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df_naics_2_state = naics_level(newDF_state, 2).reset_index(drop=True)
df_naics_3_state = naics_level(newDF_state, 3).reset_index(drop=True)
df_naics_4_state = naics_level(newDF_state, 4).reset_index(drop=True)
df_naics_5_state = naics_level(newDF_state, 5).reset_index(drop=True)
df_naics_6_state = naics_level(newDF_state, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df_naics_2_state = df_naics_2_state[df_naics_2_state.relevant_naics != '00']
df_naics_3_state = df_naics_3_state[df_naics_3_state.relevant_naics != '00']
df_naics_4_state = df_naics_4_state[df_naics_4_state.relevant_naics != '00']
df_naics_5_state = df_naics_5_state[df_naics_5_state.relevant_naics != '00']
df_naics_6_state = df_naics_6_state[df_naics_6_state.relevant_naics != '00']

In [61]:
df_naics_2_state

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
1,01,1,0,Alabama,11,"Agriculture, forestry, fishing and hunting",11,0400000US01,793.222222,6362.555556,2.458331e+05
2,01,1,0,Alabama,21,"Mining, quarrying, and oil and gas extraction",21,0400000US01,223.888889,6241.888889,4.366302e+05
3,01,1,0,Alabama,22,Utilities,22,0400000US01,383.222222,13570.888889,1.241117e+06
4,01,1,0,Alabama,23,Construction,23,0400000US01,7444.777778,80821.888889,3.913872e+06
5,01,1,0,Alabama,42,Wholesale trade,42,0400000US01,5309.222222,73602.000000,4.030520e+06
...,...,...,...,...,...,...,...,...,...,...,...
895,56,56,0,Wyoming,62,Health care and social assistance,62,0400000US56,1966.666667,32558.000000,1.500971e+06
896,56,56,0,Wyoming,71,"Arts, entertainment, and recreation",71,0400000US56,439.333333,4728.222222,9.861878e+04
897,56,56,0,Wyoming,72,Accommodation and food services,72,0400000US56,1828.111111,28071.444444,5.557270e+05
898,56,56,0,Wyoming,81,Other services (except public administration),81,0400000US56,1825.222222,8477.333333,2.752247e+05


In [62]:

states=newDF.state.unique()
#states=[13]
a=df_naics_2_state
b=df_naics_4_state
c=df_naics_6_state
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    a1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics2_state_api.tsv",sep='\t')
    #print(a1)
    #print(a1.dtypes)
    
    b1=b[b.state==state]
    b1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics4_state_api.tsv",sep='\t')
    
    c1=c[c.state==state]
    c1.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics6_state_api.tsv",sep='\t')


AL
AK
AZ
AR
CA
CO
CT
DE
FL
GA
HI
ID
IL
IN
IA
KS
KY
LA
ME
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OH
OK
OR
PA
RI
SC
SD
TN
TX
UT
VT
VA
WA
WV
WI
WY


# investigation!

In [63]:
bio_input = "113000,321113,113310,32121,32191,562213,322121,322110,"
bio_output = "325211,325991,3256,335991,325120,326190,"
green_energy = "221117,221111,221113,221114,221115,221116,221118,"
fossil_energy = "221112,324110"
cat_filter = []
cat_filter = (bio_input + bio_output + green_energy + fossil_energy).split(',')
print(cat_filter)
cat_filter2=[]
for i in cat_filter:
    #print( i[0:4])
    cat_filter2.append(i[0:4])
cat_filter2

['113000', '321113', '113310', '32121', '32191', '562213', '322121', '322110', '325211', '325991', '3256', '335991', '325120', '326190', '221117', '221111', '221113', '221114', '221115', '221116', '221118', '221112', '324110']


['1130',
 '3211',
 '1133',
 '3212',
 '3219',
 '5622',
 '3221',
 '3221',
 '3252',
 '3259',
 '3256',
 '3359',
 '3251',
 '3261',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '2211',
 '3241']

In [64]:
df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
dfp=df_naics_6[(df_naics_6.id=='13285') |  (df_naics_6.id=='13263') |  (df_naics_6.id=='13199') |  (df_naics_6.id=='13149') |  (df_naics_6.id=='13145') |  (df_naics_6.id=='13007')]
dfp=dfp[(dfp.relevant_naics.str.startswith('11')) | (dfp.relevant_naics.str.startswith('1130')) | (dfp.relevant_naics.str.startswith('3211')) |  (dfp.relevant_naics.str.startswith('1133')) | (dfp.relevant_naics.str.startswith('3212')) | (dfp.relevant_naics.str.startswith('3219')) | (dfp.relevant_naics.str.startswith('5622'))| (dfp.relevant_naics.str.startswith('3221'))| (dfp.relevant_naics.str.startswith('3252'))| (dfp.relevant_naics.str.startswith('3259'))| (dfp.relevant_naics.str.startswith('3256'))| (dfp.relevant_naics.str.startswith('3359'))| (dfp.relevant_naics.str.startswith('3251'))| (dfp.relevant_naics.str.startswith('3261'))| (dfp.relevant_naics.str.startswith('2211'))| (dfp.relevant_naics.str.startswith('3241'))]
dfp

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
245842,13007,13,7,Baker County,115114,Postharvest crop activities (except cotton gin...,11,0500000US13007,1.00,0.000,0.00
245866,13007,13,7,"Baker County, Georgia",115114,Postharvest crop activities (except cotton gin...,11,0500000US13007,1.00,0.000,0.00
282874,13145,13,145,Harris County,113210,Forest nurseries and gathering of forest products,11,0500000US13145,1.00,0.000,0.00
282875,13145,13,145,Harris County,113310,Logging,11,0500000US13145,2.00,0.000,0.00
282876,13145,13,145,Harris County,115210,Support activities for animal production,11,0500000US13145,1.00,0.000,0.00
...,...,...,...,...,...,...,...,...,...,...,...
308837,13285,13,285,"Troup County, Georgia",326113,Unlaminated plastics film and sheet (except pa...,32,0500000US13285,1.25,0.000,0.00
308838,13285,13,285,"Troup County, Georgia",326140,Polystyrene foam product manufacturing,32,0500000US13285,1.00,0.000,0.00
308839,13285,13,285,"Troup County, Georgia",326199,All other plastics product manufacturing,32,0500000US13285,2.75,169.375,6027.25
308862,13285,13,285,"Troup County, Georgia",335912,Primary battery manufacturing,33,0500000US13285,1.00,0.000,0.00


In [65]:
# dfp1=dfp.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['estab'],ascending=False).head(50)
dfp1=dfp.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['estab'],ascending=False).head(50)
dfp1

,NAICS_Sector,NAICS_TTL,state,relevant_naics,id,COUNTY,GEO_TTL,GEO_ID,estab,emp,payann
1,11,Logging,13,113310,1314513145131491314913199131991326313263132851...,2082,"Harris CountyHarris County, GeorgiaHeard Count...",0500000US131450500000US131450500000US131490500...,29.85119,115.327381,5013.809524
6,22,Electric power distribution,13,221122,131451314513149131491319913199132631328513285,1819,"Harris CountyHarris County, GeorgiaHeard Count...",0500000US131450500000US131450500000US131490500...,21.12500,19.875000,2182.500000
4,11,Support activities for forestry,13,115310,1314513145131991319913263132631328513285,1784,"Harris CountyHarris County, GeorgiaMeriwether ...",0500000US131450500000US131450500000US131990500...,12.75000,0.000000,0.000000
7,22,Fossil fuel electric power generation,13,221112,1314513149131491319913263,905,"Harris County, GeorgiaHeard CountyHeard County...",0500000US131450500000US131490500000US131490500...,10.25000,28.500000,3455.750000
5,11,Timber tract operations,13,113110,1314913149131991319913263132631328513285,1792,"Heard CountyHeard County, GeorgiaMeriwether Co...",0500000US131490500000US131490500000US131990500...,8.75000,0.000000,0.000000
9,32,All other plastics product manufacturing,13,326199,1328513285,570,"Troup CountyTroup County, Georgia",0500000US132850500000US13285,5.75000,169.375000,6027.250000
23,32,Wood container and pallet manufacturing,13,321920,13145131451328513285,860,"Harris CountyHarris County, GeorgiaTroup Count...",0500000US131450500000US131450500000US132850500...,5.25000,0.000000,0.000000
11,32,Asphalt paving mixture and block manufacturing,13,324121,13145131451328513285,860,"Harris CountyHarris County, GeorgiaTroup Count...",0500000US131450500000US131450500000US132850500...,4.25000,0.000000,0.000000
2,11,Postharvest crop activities (except cotton gin...,13,115114,13007130071319913199,412,"Baker CountyBaker County, GeorgiaMeriwether Co...",0500000US130070500000US130070500000US131990500...,4.00000,0.000000,0.000000
3,11,Support activities for animal production,13,115210,13145131451328513285,860,"Harris CountyHarris County, GeorgiaTroup Count...",0500000US131450500000US131450500000US132850500...,4.00000,0.000000,0.000000


In [66]:

df_naics_6=df_naics_6.astype({'relevant_naics': 'string'})
dfp=df_naics_6[(df_naics_6.id=='13001') |  (df_naics_6.id=='13005') |  (df_naics_6.id=='13127') |  (df_naics_6.id=='13161') |  (df_naics_6.id=='13229') |  (df_naics_6.id=='13305')]
dfp=dfp[(dfp.relevant_naics.str.startswith('11')) | (dfp.relevant_naics.str.startswith('1130')) | (dfp.relevant_naics.str.startswith('3211')) |  (dfp.relevant_naics.str.startswith('1133')) | (dfp.relevant_naics.str.startswith('3212')) | (dfp.relevant_naics.str.startswith('3219')) | (dfp.relevant_naics.str.startswith('5622'))| (dfp.relevant_naics.str.startswith('3221'))| (dfp.relevant_naics.str.startswith('3252'))| (dfp.relevant_naics.str.startswith('3259'))| (dfp.relevant_naics.str.startswith('3256'))| (dfp.relevant_naics.str.startswith('3359'))| (dfp.relevant_naics.str.startswith('3251'))| (dfp.relevant_naics.str.startswith('3261'))| (dfp.relevant_naics.str.startswith('2211'))| (dfp.relevant_naics.str.startswith('3241'))]
dfp

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
245131,13001,13,1,Appling County,113310,Logging,11,0500000US13001,11.000,61.000,2256.000
245132,13001,13,1,Appling County,115111,Cotton ginning,11,0500000US13001,1.000,0.000,0.000
245133,13001,13,1,Appling County,221113,Nuclear electric power generation,22,0500000US13001,1.000,0.000,0.000
245134,13001,13,1,Appling County,221122,Electric power distribution,22,0500000US13001,6.000,0.000,0.000
245147,13001,13,1,Appling County,321113,Sawmills,32,0500000US13001,3.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...
312719,13305,13,305,"Wayne County, Georgia",221122,Electric power distribution,22,0500000US13305,3.875,11.125,666.375
312745,13305,13,305,"Wayne County, Georgia",321113,Sawmills,32,0500000US13305,1.000,0.000,0.000
312746,13305,13,305,"Wayne County, Georgia",321999,All other miscellaneous wood product manufactu...,32,0500000US13305,2.000,0.000,0.000
312747,13305,13,305,"Wayne County, Georgia",322110,Pulp mills,32,0500000US13305,1.000,0.000,0.000


In [67]:
# dfp1=dfp.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['payann'],ascending=False).head(50)
dfp1=dfp.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().sort_values(by=['payann'],ascending=False).head(50)
dfp1

,NAICS_Sector,NAICS_TTL,state,relevant_naics,id,COUNTY,GEO_TTL,GEO_ID,estab,emp,payann
3,11,Logging,13,113310,1300113001130051300513127131271316113161132291...,1656,"Appling CountyAppling County, GeorgiaBacon Cou...",0500000US130010500000US130010500000US130050500...,110.625000,634.500,26069.625
21,32,Sawmills,13,321113,130011300113127131271316113161132291322913305,1341,"Appling CountyAppling County, GeorgiaGlynn Cou...",0500000US130010500000US130010500000US131270500...,22.825000,134.900,7299.150
8,22,Electric power distribution,13,221122,1300113001130051300513127131271316113161132291...,1656,"Appling CountyAppling County, GeorgiaBacon Cou...",0500000US130010500000US130010500000US130050500...,47.250000,58.375,5359.000
4,11,Postharvest crop activities (except cotton gin...,13,115114,130051300513161,171,"Bacon CountyBacon County, GeorgiaJeff Davis Co...",0500000US130050500000US130050500000US13161,7.875000,33.250,1480.125
6,11,Support activities for forestry,13,115310,1300113005130051312713161132291322913305,1062,"Appling County, GeorgiaBacon CountyBacon Count...",0500000US130010500000US130050500000US130050500...,12.000000,8.000,283.500
0,11,Cotton ginning,13,115111,13001130011316113161,324,"Appling CountyAppling County, GeorgiaJeff Davi...",0500000US130010500000US130010500000US131610500...,4.000000,0.000,0.000
16,32,"Cyclic crude, intermediate, and gum and wood c...",13,325194,1312713127,254,"Glynn CountyGlynn County, Georgia",0500000US131270500000US13127,2.000000,0.000,0.000
26,32,Wood window and door manufacturing,13,321911,1300113001,2,"Appling CountyAppling County, Georgia",0500000US130010500000US13001,2.000000,0.000,0.000
25,32,Wood preservation,13,321114,1322913229,458,"Pierce CountyPierce County, Georgia",0500000US132290500000US13229,2.000000,0.000,0.000
24,32,Wood container and pallet manufacturing,13,321920,130051316113161,327,"Bacon County, GeorgiaJeff Davis CountyJeff Dav...",0500000US130050500000US131610500000US13161,3.000000,0.000,0.000


## investigating the state total vs aggregation of counties

In [68]:
#states=newDF.state.unique()
states=[13]
a=state_level(df_naics_2_state)
b=state_level(df_naics_4_state)
c=state_level(df_naics_6_state)
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a1=a[a.state==state]
    
    b1=b[b.state==state]
    
    c1=c[c.state==state]

GA


In [69]:
a1.sort_values(by=['payann'],ascending=False).head(50)

,id,state,COUNTY,GEO_TTL,relevant_naics,NAICS_TTL,NAICS_Sector,GEO_ID,estab,emp,payann
175,13,13,0,Georgia,62,Health care and social assistance,62,0400000US13,24432.333333,488066.444444,2.325001e+07
171,13,13,0,Georgia,54,"Professional, scientific, and technical services",54,0400000US13,29194.555556,256054.333333,1.886882e+07
169,13,13,0,Georgia,52,Finance and insurance,52,0400000US13,14869.444444,174085.000000,1.439456e+07
172,13,13,0,Georgia,55,Management of companies and enterprises,55,0400000US13,1796.444444,128713.888889,1.355605e+07
167,13,13,0,Georgia,42,Wholesale trade,42,0400000US13,13076.444444,202360.111111,1.282881e+07
173,13,13,0,Georgia,56,Administrative and support and waste managemen...,56,0400000US13,12418.777778,370034.777778,1.280819e+07
168,13,13,0,Georgia,51,Information,51,0400000US13,4372.333333,124068.222222,1.094533e+07
166,13,13,0,Georgia,23,Construction,23,0400000US13,17284.000000,165330.888889,8.784317e+06
177,13,13,0,Georgia,72,Accommodation and food services,72,0400000US13,20259.666667,412345.555556,6.674711e+06
178,13,13,0,Georgia,81,Other services (except public administration),81,0400000US13,21286.666667,157285.555556,4.514646e+06


In [70]:
states=[13]

b1=county_level(df_naics_2)
c1=b1[b1.state==13]
c1.astype({'NAICS_Sector': 'int'})
# d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d1=c1.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d1.insert(0, 'id', 13)
d1.insert(1, 'COUNTY', 999)
d1.insert(2, 'GEO_TTL', 'Statewide')

b2=county_level(df_naics_4)
c2=b2[b2.state==13]
c2.astype({'NAICS_Sector': 'int'})
# d2=c2.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d2=c2.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d2.insert(0, 'id', 13)
d2.insert(1, 'COUNTY', 999)
d2.insert(2, 'GEO_TTL', 'Statewide')

b3=county_level(df_naics_6)
c3=b3[b3.state==13]
c3.astype({'NAICS_Sector': 'int'})
# d3=c3.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d3=c3.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
d3.insert(0, 'id', 13)
d3.insert(1, 'COUNTY', 999)
d3.insert(2, 'GEO_TTL', 'Statewide')

for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
    print(stateName)
    a11=d1[d1.state==state]

    
    b11=d2[d2.state==state]
    
    c11=d3[d3.state==state]


GA


In [71]:
a11.sort_values(by=['payann'],ascending=False).head(50)

,id,COUNTY,GEO_TTL,NAICS_Sector,NAICS_TTL,state,relevant_naics,GEO_ID,estab,emp,payann
12,13,999,Statewide,62,Health care and social assistance,13,62,0500000US130010500000US130010500000US130030500...,47770.916667,947398.541667,4.437330e+07
8,13,999,Statewide,54,"Professional, scientific, and technical services",13,54,0500000US130010500000US130010500000US130030500...,57553.535714,468308.178571,3.476489e+07
6,13,999,Statewide,52,Finance and insurance,13,52,0500000US130010500000US130010500000US130030500...,29119.732143,308902.500000,2.556367e+07
9,13,999,Statewide,55,Management of companies and enterprises,13,55,0500000US130010500000US130050500000US130050500...,3519.386905,238169.659524,2.485900e+07
4,13,999,Statewide,42,Wholesale trade,13,42,0500000US130010500000US130010500000US130030500...,25821.384524,376226.086905,2.394545e+07
5,13,999,Statewide,51,Information,13,51,0500000US130010500000US130010500000US130050500...,8468.169048,227611.854762,1.999248e+07
10,13,999,Statewide,56,Administrative and support and waste managemen...,13,56,0500000US130010500000US130010500000US130030500...,24093.922619,514989.660714,1.836583e+07
3,13,999,Statewide,23,Construction,13,23,0500000US130010500000US130010500000US130030500...,33849.103571,308075.289286,1.601481e+07
14,13,999,Statewide,72,Accommodation and food services,13,72,0500000US130010500000US130010500000US130030500...,39579.255952,781142.101190,1.230098e+07
15,13,999,Statewide,81,Other services (except public administration),13,81,0500000US130010500000US130010500000US130030500...,42009.257143,308012.492857,8.610047e+06


# Estimation for 0 values

In [72]:
#newDF=newDF.astype({'fips': 'int'})
# df=df.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
df=df.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#b=b[["id","relevant_naics","emp","payann","estab"]]
df

,GEO_ID,GEO_TTL,COUNTY,YEAR,relevant_naics,NAICS_TTL,estab,emp,payann,state,NAICS_Sector,id
0,0500000US08113,San Miguel County,113,2012,00,Total for all sectors,622,4291,116855,8,00,08113
1,0500000US08113,San Miguel County,113,2012,11,"Agriculture, forestry, fishing and hunting",3,0,0,8,11,08113
2,0500000US08113,San Miguel County,113,2012,114,"Fishing, hunting and trapping",1,0,0,8,11,08113
3,0500000US08113,San Miguel County,113,2012,1141,Fishing,1,0,0,8,11,08113
4,0500000US08113,San Miguel County,113,2012,11411,Fishing,1,0,0,8,11,08113
...,...,...,...,...,...,...,...,...,...,...,...,...
2876,0500000US10001,"Kent County, Delaware",1,2020,81392,Professional organizations,4,12,273,10,81,10001
2877,0500000US10001,"Kent County, Delaware",1,2020,813920,Professional organizations,4,12,273,10,81,10001
2878,0500000US10001,"Kent County, Delaware",1,2020,81393,Labor unions and similar labor organizations,16,42,3157,10,81,10001
2879,0500000US10001,"Kent County, Delaware",1,2020,813930,Labor unions and similar labor organizations,16,42,3157,10,81,10001


In [73]:
# df=df.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
df=df.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})

df1_naics_2 = naics_level(df, 2).reset_index(drop=True)
df1_naics_3 = naics_level(df, 3).reset_index(drop=True)
df1_naics_4 = naics_level(df, 4).reset_index(drop=True)
df1_naics_5 = naics_level(df, 5).reset_index(drop=True)
df1_naics_6 = naics_level(df, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df1_naics_2 = df1_naics_2[df1_naics_2.relevant_naics != '00']
df1_naics_3 = df1_naics_3[df1_naics_3.relevant_naics != '00']
df1_naics_4 = df1_naics_4[df1_naics_4.relevant_naics != '00']
df1_naics_5 = df1_naics_5[df1_naics_5.relevant_naics != '00']
df1_naics_6 = df1_naics_6[df1_naics_6.relevant_naics != '00']

In [74]:
# df_state=df_state.rename(columns={"fips": "id", "NAICS2012": "relevant_naics","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
df_state=df_state.rename(columns={"fips": "id","EMP":"emp","PAYANN":"payann","ESTAB":"estab"})
#df.head()
# df_county['NAICS2012'].apply(lambda v: v.split('-')[0])

df1_naics_2_state = naics_level(df_state, 2).reset_index(drop=True)
df1_naics_3_state = naics_level(df_state, 3).reset_index(drop=True)
df1_naics_4_state = naics_level(df_state, 4).reset_index(drop=True)
df1_naics_5_state = naics_level(df_state, 5).reset_index(drop=True)
df1_naics_6_state = naics_level(df_state, 6).reset_index(drop=True)

# First remove all the per-county totals across all sectors
#df_totals = df_naics_2[df_naics_2.NAICS2012 == '00']
df1_naics_2_state = df1_naics_2_state[df1_naics_2_state.relevant_naics != '00']
df1_naics_3_state = df1_naics_3_state[df1_naics_3_state.relevant_naics != '00']
df1_naics_4_state = df1_naics_4_state[df1_naics_4_state.relevant_naics != '00']
df1_naics_5_state = df1_naics_5_state[df1_naics_5_state.relevant_naics != '00']
df1_naics_6_state = df1_naics_6_state[df1_naics_6_state.relevant_naics != '00']

In [75]:
states=df.state.unique()
#states=[13]
naics=[2,4,6]
#years=[2012]
endyear = 2021
years=range(2012,endyear)
#naics=[2]
for state in states:
    stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0] 
    for i in naics:
        print(stateName+str(i))
        allData_ets1=[]
        allData_ets3=[]
        for year in years:
            a=f'df1_naics_{i}_state'
            a=globals()[a]
            a=a[a.state==state]
            everything=a[a.YEAR==year]
            #print("everything")
            #print(everything.shape)
            b=f'df1_naics_{i}'
            b=globals()[b]
            b1=county_level(b)
            c1=b1[b1.state==state]
            c1=c1[c1.YEAR==year]
            c1.astype({'NAICS_Sector': 'int'})
            # d1=c1.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics','YEAR'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
            d1=c1.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics','YEAR'],as_index=False).sum().drop(["COUNTY", "id", "GEO_TTL"],axis=1)
            d1.insert(0, 'id', state)
            d1.insert(1, 'COUNTY', 999)
            d1.insert(2, 'GEO_TTL', 'Statewide')
            known=d1
            countylevel=c1
            #print("countylevel")
            #print(countylevel.shape)
            valueDif=everything.merge(known[['relevant_naics','estab','emp','payann']], on='relevant_naics', how='inner')
            valueDif['payann_diff']=valueDif['payann_x']-valueDif['payann_y']
            valueDif['emp_diff']=valueDif['emp_x']-valueDif['emp_y']
            valueDif['estab_diff']=valueDif['estab_x']-valueDif['estab_y']
            
            payannemp0=countylevel[(countylevel['payann']==0) & (countylevel['emp']==0)]
            #print(payannemp0.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics','YEAR'],as_index=False).sum())
            #print('XXXXXXXXXXXXX')
            #print(valueDif)
            #print("os")
            #print(payannemp0.shape)
            middle0=payannemp0.drop(['state','GEO_TTL','NAICS_Sector','emp','payann','id','COUNTY'],axis=1)
            # estab0s=middle0.groupby(['relevant_naics','NAICS2012_TTL'],as_index=False).sum()
            estab0s=middle0.groupby(['relevant_naics','NAICS_TTL'],as_index=False).sum()
            estab0s=estab0s.rename(columns={'estab':'total_estabs'})
            merged1=payannemp0.merge(estab0s[['relevant_naics','total_estabs']], on='relevant_naics', how='left')
            merged1['estab_ratio']=merged1['estab']/merged1['total_estabs']
            merged2=merged1.merge(valueDif[['relevant_naics','payann_diff','emp_diff']], on='relevant_naics', how='left')
            merged2=merged2.drop(['payann','emp'],axis=1)
            merged2['payann']=merged2['payann_diff']*merged2['estab_ratio']
            #(state total minus the known county-totals) * (establishments in county / establishments for all counties without county values)
            #for establishments for all counties without county values:
            #first:find the county level data that have payann and emp of 0, and use their number of establishment
            merged2['emp']=merged2['emp_diff']*merged2['estab_ratio']
            merged2=merged2.drop(['total_estabs','estab_ratio','payann_diff','emp_diff'],axis=1)
            #print("merges2")
            #print(merged2.shape)
            merged3=payannemp0.merge(valueDif[['relevant_naics','payann_diff','emp_diff','estab_diff']], on='relevant_naics', how='left')
            merged3=merged3.drop(['payann','emp'],axis=1)
            merged3['estab_scale']=merged3['estab']/merged3['estab_diff']
            merged3['payann']=merged3['payann_diff']*merged3['estab_scale']
            merged3['emp']=merged3['emp_diff']*merged3['estab_scale']
            merged3.loc[merged3['emp'] <0, 'emp'] = 0
            #second:find the number of establishments by taking the difference of state-level data from api, and the aggregation of county-level data.
            #this one gives 0s for situations where the difference in number of estabs is less than 0.
            merged3.loc[merged3['payann'] <0, 'payann'] = 0
            merged3=merged3.drop(['payann_diff','emp_diff','estab_diff','estab_scale'],axis=1)
            #third:state total * (establishments in county / establishments for entire state)
            payannemp0s=payannemp0.drop(['emp','payann'],axis=1)
            merged4=payannemp0s.merge(everything[['relevant_naics','estab','emp','payann']], on='relevant_naics', how='left')
            merged4['estab_scale']=merged4['estab_x']/merged4['estab_y']
            merged4['payann']=merged4['payann']*merged4['estab_scale']
            merged4['emp']=merged4['emp']*merged4['estab_scale']
            merged4=merged4.rename(columns={'estab_x':'estab'})
            merged4=merged4.drop(['estab_y','estab_scale'],axis=1)

            payannemppos=countylevel[(countylevel['payann']>0) | (countylevel['emp']>0)]
            
            #print("non os")
            #print(payannemppos.shape)
            merged2['estimate']=1
            #merged4['estimate']=1

            
            est1=pd.concat([payannemppos,merged2])
            
            #est2=pd.concat([payannemppos,merged3])
            est3=pd.concat([payannemppos,merged4])
            
            est3=est3.merge(countylevel[['relevant_naics','emp','payann']], on='relevant_naics', how='left')
            est3.loc[(est3['payann_x'] != est3['payann_y']) | (est3['emp_x'] != est3['emp_x']), 'estimate'] = 1
            est3=est3.drop(['payann_y','emp_y'],axis=1)
            est3=est3.rename(columns={"payann_x": "payann", "emp_x":"emp"})
            #print(est3)
            est1.loc[est1['estimate'] !=1, 'estimate'] = 0
            est3.loc[est3['estimate'] !=1, 'estimate'] = 0
            allData_ets1.append(est1)
            allData_ets3.append(est3)
            #print(est1)
            ###est1.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est1.tsv",sep='\t')
            #est2.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est2.tsv",sep='\t')
            ###est3.to_csv(f"usa/{stateName}/industries_state{state}_naics{i}_est3.tsv",sep='\t')
        for j in range(0,len(years)):
            if j==0:
                est1_all=allData_ets1[j]
                est3_all=allData_ets3[j]
            else:
                est1_all=pd.concat([est1_all,allData_ets1[j]])
                est3_all=pd.concat([est1_all,allData_ets3[j]])
        # est1_final=(est1_all.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics','COUNTY','id','GEO_TTL','GEO_ID'],as_index=False).mean()).drop('YEAR',axis=1)
        est1_final=(est1_all.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics','COUNTY','id','GEO_TTL','GEO_ID'],as_index=False).mean()).drop('YEAR',axis=1)
        # est3_final=(est3_all.groupby(['NAICS_Sector','NAICS2012_TTL','state','relevant_naics','COUNTY','id','GEO_TTL','GEO_ID'],as_index=False).mean()).drop('YEAR',axis=1)
        est3_final=(est3_all.groupby(['NAICS_Sector','NAICS_TTL','state','relevant_naics','COUNTY','id','GEO_TTL','GEO_ID'],as_index=False).mean()).drop('YEAR',axis=1)
        #print(est3_final)
        est1_final=est1_final.astype({'estimate': 'float'})
        est3_final=est3_final.astype({'estimate': 'float'})
        est1_final.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est1.tsv",sep='\t')
        est3_final.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est3.tsv",sep='\t')
        #print("YYYYYYYYYYYYYYYYYYY")
        #print(est1_final)

CO2
CO4
CO6
SD2
SD4
SD6
NC2
NC4
NC6
DE2
DE4
DE6
AR2
AR4
AR6
RI2
RI4
RI6
VA2
VA4
VA6
KS2
KS4
KS6
GA2
GA4
GA6
WY2
WY4
WY6
WI2
WI4
WI6
VT2
VT4
VT6
UT2
UT4
UT6
NY2
NY4
NY6
SC2
SC4
SC6
AK2
AK4
AK6
LA2
LA4
LA6
MA2
MA4
MA6
TN2
TN4
TN6
PA2
PA4
PA6
MN2
MN4
MN6
CA2
CA4
CA6
AZ2
AZ4
AZ6
MI2
MI4
MI6
ND2
ND4
ND6
ID2
ID4
ID6
TX2
TX4
TX6
IA2
IA4
IA6
CT2
CT4
CT6
WA2
WA4
WA6
WV2
WV4
WV6
MT2
MT4
MT6
ME2
ME4
ME6
IN2
IN4
IN6
OR2
OR4
OR6
NJ2
NJ4
NJ6
IL2
IL4
IL6
MD2
MD4
MD6
MS2
MS4
MS6
AL2
AL4
AL6
OH2
OH4
OH6
FL2
FL4
FL6
NE2
NE4
NE6
NH2
NH4
NH6
KY2
KY4
KY6
MO2
MO4
MO6
NM2
NM4
NM6
OK2
OK4
OK6
HI2
HI4
HI6
NV2
NV4
NV6


## Merging CSVs

In [76]:
states=df.state.unique()
#states=[13]
naics=[2,4,6]
#naics=[6]
for state in states:
    for i in naics: 
        stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
        print(stateName+str(i))
        countylevel=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}.tsv",sep='\t')
        est1=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est1.tsv",sep='\t')
        est3=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est3.tsv",sep='\t')
        merged=countylevel.merge(est1[['relevant_naics','COUNTY','emp','payann','estimate']], on=['relevant_naics','COUNTY'], how='left').drop('Unnamed: 0',axis=1)
        merged2=merged.merge(est3[['relevant_naics','COUNTY','emp','payann','estimate']], on=['relevant_naics','COUNTY'], how='left')
        merged2=merged2.rename(columns={"emp_x": "emp_reported","payann_x":"payann_reported", "emp_y": "emp_est1","payann_y":"payann_est1","estimate_x":"estimate_est1", "emp": "emp_est3","payann":"payann_est3","estimate_y":"estimate_est3"})
        #print(merged2.dtypes)
        merged2.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_all.tsv",sep='\t')
        
        #statelevel
        statelev=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state.tsv",sep='\t')
        statelev_api=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state_api.tsv",sep='\t')
        #print(statelev_api)
        merged3=statelev.merge(statelev_api[['relevant_naics','emp','payann','estab']], on=['relevant_naics'], how='left').drop('Unnamed: 0',axis=1)
        merged4=merged3.rename(columns={"emp_x": "emp_agg","payann_x":"payann_agg","estab_x":"estab_agg", "emp_y": "emp_api","payann_y":"payann_api","estab_y":"estab_api"})
        #print(merged4)
        merged4.to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state_all.tsv",sep='\t')
        
        os.remove(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}.tsv")
        os.remove(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est1.tsv")
        os.remove(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_est3.tsv")
        os.remove(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state.tsv")
        os.remove(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state_api.tsv")

CO2
CO4
CO6
SD2
SD4
SD6
NC2
NC4
NC6
DE2
DE4
DE6
AR2
AR4
AR6
RI2
RI4
RI6
VA2
VA4
VA6
KS2
KS4
KS6
GA2
GA4
GA6
WY2
WY4
WY6
WI2
WI4
WI6
VT2
VT4
VT6
UT2
UT4
UT6
NY2
NY4
NY6
SC2
SC4
SC6
AK2
AK4
AK6
LA2
LA4
LA6
MA2
MA4
MA6
TN2
TN4
TN6
PA2
PA4
PA6
MN2
MN4
MN6
CA2
CA4
CA6
AZ2
AZ4
AZ6
MI2
MI4
MI6
ND2
ND4
ND6
ID2
ID4
ID6
TX2
TX4
TX6
IA2
IA4
IA6
CT2
CT4
CT6
WA2
WA4
WA6
WV2
WV4
WV6
MT2
MT4
MT6
ME2
ME4
ME6
IN2
IN4
IN6
OR2
OR4
OR6
NJ2
NJ4
NJ6
IL2
IL4
IL6
MD2
MD4
MD6
MS2
MS4
MS6
AL2
AL4
AL6
OH2
OH4
OH6
FL2
FL4
FL6
NE2
NE4
NE6
NH2
NH4
NH6
KY2
KY4
KY6
MO2
MO4
MO6
NM2
NM4
NM6
OK2
OK4
OK6
HI2
HI4
HI6
NV2
NV4
NV6


# finding industries with 0 for both state and county level

In [77]:
states=df.state.unique()
#states=[13]
naics=[2,4,6]
#naics=[6]
for state in states:
    for i in naics: 
        stateName=stateFips.loc[stateFips.FIPS==state,"Postal Code"].values[0]
        print(stateName+str(i))
        api=pd.read_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_state_all.tsv",sep='\t')
        api_0s=api[(api.emp_agg==0) & (api.payann_agg==0) & (api.emp_api==0) & (api.payann_api==0)]
        # api_0s[['id','COUNTY','GEO_TTL','NAICS_Sector','NAICS2012_TTL','state','relevant_naics','estab_agg','estab_api']].to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_0s.tsv",sep='\t')
        api_0s[['id','COUNTY','GEO_TTL','NAICS_Sector','NAICS_TTL','state','relevant_naics','estab_agg','estab_api']].to_csv(f"../../../us/state-new/{stateName}/industries_state{state}_naics{i}_0s.tsv",sep='\t')

CO2
CO4
CO6
SD2
SD4
SD6
NC2
NC4
NC6
DE2
DE4
DE6
AR2
AR4
AR6
RI2
RI4
RI6
VA2
VA4
VA6
KS2
KS4
KS6
GA2
GA4
GA6
WY2
WY4
WY6
WI2
WI4
WI6
VT2
VT4
VT6
UT2
UT4
UT6
NY2
NY4
NY6
SC2
SC4
SC6
AK2
AK4
AK6
LA2
LA4
LA6
MA2
MA4
MA6
TN2
TN4
TN6
PA2
PA4
PA6
MN2
MN4
MN6
CA2
CA4
CA6
AZ2
AZ4
AZ6
MI2
MI4
MI6
ND2
ND4
ND6
ID2
ID4
ID6
TX2
TX4
TX6
IA2
IA4
IA6
CT2
CT4
CT6
WA2
WA4
WA6
WV2
WV4
WV6
MT2
MT4
MT6
ME2
ME4
ME6
IN2
IN4
IN6
OR2
OR4
OR6
NJ2
NJ4
NJ6
IL2
IL4
IL6
MD2
MD4
MD6
MS2
MS4
MS6
AL2
AL4
AL6
OH2
OH4
OH6
FL2
FL4
FL6
NE2
NE4
NE6
NH2
NH4
NH6
KY2
KY4
KY6
MO2
MO4
MO6
NM2
NM4
NM6
OK2
OK4
OK6
HI2
HI4
HI6
NV2
NV4
NV6
